In [1]:
from rdflib import Graph
import pandas as pd
import ssl

#ctx = ssl.create_default_context()
#ctx.check_hostname = False
#ctx.verify_mode = ssl.CERT_NONE

ssl._create_default_https_context = ssl._create_unverified_context

## 1te Abfrage: Gemeinde, Gemeindeeinwohnerzahl, Kanton, Kantonseinwohnerzahl

In [2]:
GemeindeKantone_QUERY = '''
PREFIX schema: <http://schema.org/>
PREFIX gn: <http://www.geonames.org/ontology#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?PLZ ?Gemeinde ?Name ?Population ?Cantonname ?Cantonpopul
WHERE {
        SERVICE <https://geo.ld.admin.ch/query>
            {   ?Gemeinde gn:featureCode gn:A.ADM3 .
                ?Gemeinde schema:name ?Name .
                ?Gemeinde gn:population ?Population .
                ?Gemeinde <http://purl.org/dc/terms/issued> ?Date .

                ?Gemeinde gn:parentADM1 ?Canton .
                ?Canton schema:name ?Cantonname .
                ?Canton gn:population ?Cantonpopul .

                OPTIONAL {?Gemeinde gn:postalCode ?PLZ}.

                FILTER (?Date = "2023-01-01"^^xsd:date)
                }
            }
            ORDER BY DESC(?Population)

        '''

# Graph erstellen und SPARQL-Abfrage durchführen
g = Graph()
results = g.query(GemeindeKantone_QUERY)

result_list = []
for row in results:
    result_list.append(row)

# Liste von Tupeln in ein Pandas DataFrame umwandeln
df_g = pd.DataFrame(result_list, columns=['plz', 'gemeinde', 'name', 'population', 'cantonname', 'cantonpopul'])

# DataFrame anzeigen
#print(df_g)

# 2te Abfrage: Kanton und deren Abkürzungen

In [3]:
KurznamenKanton_QUERY = '''
PREFIX schema: <http://schema.org/>

select  distinct ?Canton ?Kurzname ?Cantonname
where {
        SERVICE <https://geo.ld.admin.ch/query>
            {?Canton schema:containedInPlace ?Land.
            ?Canton schema:name ?Cantonname .
            ?Canton <http://schema.org/alternateName> ?Kurzname

        FILTER (lang(?Cantonname) ='de' || lang(?Cantonname) ='fr' || lang(?Cantonname) ='it')
}}
'''
g2 = Graph()
results_g2 = g2.query(KurznamenKanton_QUERY)

result_g2_list = []
for row in results_g2:
    result_g2_list.append(row)

# Liste von Tupeln in ein Pandas DataFrame umwandeln
df_g2 = pd.DataFrame(result_g2_list, columns=['canton', 'kurzname', 'cantonname',])

# DataFrame anzeigen
#print(df_g2)

In [4]:
df_g['cantonname'] = df_g['cantonname'].str.replace('\s', '', regex=True)
df_g2['cantonname'] = df_g2['cantonname'].str.replace('\s', '', regex=True)

In [8]:
df_g2

,canton,kurzname,cantonname
0,https://geo.ld.admin.ch/biota/cantonregion/1,ZH,Zürich
1,https://geo.ld.admin.ch/biota/cantonregion/1,ZH,Zurich
2,https://geo.ld.admin.ch/biota/cantonregion/1,ZH,Zurigo
3,https://geo.ld.admin.ch/biota/cantonregion/2,BE,Bern
4,https://geo.ld.admin.ch/biota/cantonregion/2,BE,Berne
...,...,...,...
70,https://geo.ld.admin.ch/biota/cantonregion/25,GE,Genève
71,https://geo.ld.admin.ch/biota/cantonregion/25,GE,Ginevra
72,https://geo.ld.admin.ch/biota/cantonregion/26,JU,Giura
73,https://geo.ld.admin.ch/biota/cantonregion/26,JU,Jura


In [10]:
df_merge = pd.merge(df_g, df_g2, on='cantonname', how='left')

df_merge


,plz,gemeinde,name,population,cantonname,cantonpopul,canton,kurzname
0,None,https://geo.ld.admin.ch/boundaries/municipalit...,Zürich,423193,Zürich,1564662,https://geo.ld.admin.ch/biota/cantonregion/1,ZH
1,None,https://geo.ld.admin.ch/boundaries/municipalit...,Genève,203401,Genève,509448,https://geo.ld.admin.ch/biota/cantonregion/25,GE
2,None,https://geo.ld.admin.ch/boundaries/municipalit...,Basel,173064,Basel-Stadt,196036,NaN,NaN
3,None,https://geo.ld.admin.ch/boundaries/municipalit...,Lausanne,140619,Vaud,822968,https://geo.ld.admin.ch/biota/cantonregion/22,VD
4,None,https://geo.ld.admin.ch/boundaries/municipalit...,Lausanne,140619,Vaud,822968,https://geo.ld.admin.ch/biota/cantonregion/22,VD
...,...,...,...,...,...,...,...,...
2547,None,https://geo.ld.admin.ch/boundaries/municipalit...,Rebévelier,41,Bern,1047422,https://geo.ld.admin.ch/biota/cantonregion/2,BE
2548,None,https://geo.ld.admin.ch/boundaries/municipalit...,Cerentino,36,Ticino,352181,https://geo.ld.admin.ch/biota/cantonregion/21,TI
2549,None,https://geo.ld.admin.ch/boundaries/municipalit...,Schelten,34,Bern,1047422,https://geo.ld.admin.ch/biota/cantonregion/2,BE
2550,None,https://geo.ld.admin.ch/boundaries/municipalit...,Kammersrohr,33,Solothurn,280245,https://geo.ld.admin.ch/biota/cantonregion/11,SO


In [11]:
df_merge['kurzname'].mask(df_merge['cantonname'] == 'Basel-Stadt', 'BS', inplace=True)
df_merge['kurzname'].mask(df_merge['cantonname'] == 'Basel-Landschaft', 'BL', inplace=True)
df_merge

,plz,gemeinde,name,population,cantonname,cantonpopul,canton,kurzname
0,None,https://geo.ld.admin.ch/boundaries/municipalit...,Zürich,423193,Zürich,1564662,https://geo.ld.admin.ch/biota/cantonregion/1,ZH
1,None,https://geo.ld.admin.ch/boundaries/municipalit...,Genève,203401,Genève,509448,https://geo.ld.admin.ch/biota/cantonregion/25,GE
2,None,https://geo.ld.admin.ch/boundaries/municipalit...,Basel,173064,Basel-Stadt,196036,NaN,BS
3,None,https://geo.ld.admin.ch/boundaries/municipalit...,Lausanne,140619,Vaud,822968,https://geo.ld.admin.ch/biota/cantonregion/22,VD
4,None,https://geo.ld.admin.ch/boundaries/municipalit...,Lausanne,140619,Vaud,822968,https://geo.ld.admin.ch/biota/cantonregion/22,VD
...,...,...,...,...,...,...,...,...
2547,None,https://geo.ld.admin.ch/boundaries/municipalit...,Rebévelier,41,Bern,1047422,https://geo.ld.admin.ch/biota/cantonregion/2,BE
2548,None,https://geo.ld.admin.ch/boundaries/municipalit...,Cerentino,36,Ticino,352181,https://geo.ld.admin.ch/biota/cantonregion/21,TI
2549,None,https://geo.ld.admin.ch/boundaries/municipalit...,Schelten,34,Bern,1047422,https://geo.ld.admin.ch/biota/cantonregion/2,BE
2550,None,https://geo.ld.admin.ch/boundaries/municipalit...,Kammersrohr,33,Solothurn,280245,https://geo.ld.admin.ch/biota/cantonregion/11,SO


In [30]:
# Dict erstellen für Kantons-Url & Kurzname
import rdflib

kanton_dict_geo = df_g2.set_index('kurzname')['canton'].to_dict()

# Eintrag löschen
del kanton_dict_geo[rdflib.term.Literal('BL/BS')]

# Zwei neue Einträge hinzufügen
kanton_dict_geo[rdflib.term.Literal('BS')] = rdflib.term.URIRef('https://geo.ld.admin.ch/biota/cantonregion/12')
kanton_dict_geo[rdflib.term.Literal('BL')] = rdflib.term.URIRef('https://geo.ld.admin.ch/biota/cantonregion/13')
kanton_dict_geo[rdflib.term.Literal('CH')] = rdflib.term.URIRef('https://geo.ld.admin.ch/biota/cantonregion/CHE')
kanton_dict_geo

{rdflib.term.Literal('ZH'): rdflib.term.URIRef('https://geo.ld.admin.ch/biota/cantonregion/1'),
 rdflib.term.Literal('BE'): rdflib.term.URIRef('https://geo.ld.admin.ch/biota/cantonregion/2'),
 rdflib.term.Literal('LU'): rdflib.term.URIRef('https://geo.ld.admin.ch/biota/cantonregion/3'),
 rdflib.term.Literal('UR'): rdflib.term.URIRef('https://geo.ld.admin.ch/biota/cantonregion/4'),
 rdflib.term.Literal('SZ'): rdflib.term.URIRef('https://geo.ld.admin.ch/biota/cantonregion/5'),
 rdflib.term.Literal('OW'): rdflib.term.URIRef('https://geo.ld.admin.ch/biota/cantonregion/6'),
 rdflib.term.Literal('NW'): rdflib.term.URIRef('https://geo.ld.admin.ch/biota/cantonregion/7'),
 rdflib.term.Literal('GL'): rdflib.term.URIRef('https://geo.ld.admin.ch/biota/cantonregion/8'),
 rdflib.term.Literal('ZG'): rdflib.term.URIRef('https://geo.ld.admin.ch/biota/cantonregion/9'),
 rdflib.term.Literal('FR'): rdflib.term.URIRef('https://geo.ld.admin.ch/biota/cantonregion/10'),
 rdflib.term.Literal('SO'): rdflib.term

# 3te Abfrage: Anzahl Gesuche pro Jahr und Region für Förderprogramm

In [28]:
Gutsprachen_QUERY = '''
PREFIX schema: <http://schema.org/>

select distinct ?kurzname ?jahr ?anzahl
where {
        SERVICE <https://lindas.admin.ch/query>
            {?s schema:eligibleRegion ?kurzname.
            ?s <https://energy.ld.admin.ch/sfoe/bfe_ogd18_gebaeudeprogramm_anzahl_gesuche/Jahr> ?jahr.
            ?s <https://energy.ld.admin.ch/sfoe/bfe_ogd18_gebaeudeprogramm_anzahl_gesuche/anzahl-gesuche-mit-auszahlungen> ?anzahl.
}}
'''

# Graph erstellen und SPARQL-Abfrage durchführen
g3 = Graph()
results_g3 = g3.query(Gutsprachen_QUERY)

result_g3_list = []
for row in results_g3:
    result_g3_list.append(row)

# Liste in ein Pandas DataFrame umwandeln
df_g3 = pd.DataFrame(result_g3_list, columns=['kurzname', 'jahr', 'anzahl'])



In [29]:
# DataFrame anzeigen
df_g3

,kurzname,jahr,anzahl
0,CH,2010,0
1,CH,2010,5514
2,CH,2011,0
3,CH,2011,20547
4,CH,2012,0
...,...,...,...
359,https://ld.admin.ch/canton/4,2022,147
360,https://ld.admin.ch/canton/22,2022,2151
361,https://ld.admin.ch/canton/23,2022,1277
362,https://ld.admin.ch/canton/9,2022,104


In [32]:
kanton_dict_lindas ={
    '^https://ld.admin.ch/canton/1$' : 'ZH',
    '^https://ld.admin.ch/canton/2$' : 'BE',
    '^https://ld.admin.ch/canton/3$' : 'LU',
    '^https://ld.admin.ch/canton/4$' : 'UR',
    '^https://ld.admin.ch/canton/5$' : 'SZ',
    '^https://ld.admin.ch/canton/6$' : 'OW',
    '^https://ld.admin.ch/canton/7$' : 'NW',
    '^https://ld.admin.ch/canton/8$' : 'GL',
    '^https://ld.admin.ch/canton/9$' : 'ZG',
    '^https://ld.admin.ch/canton/10$' : 'FR',
    '^https://ld.admin.ch/canton/11$' : 'SO',
    '^https://ld.admin.ch/canton/12$' : 'BS',
    '^https://ld.admin.ch/canton/13$' : 'BL',
    '^https://ld.admin.ch/canton/14$' : 'SH',
    '^https://ld.admin.ch/canton/15$' : 'AR',
    '^https://ld.admin.ch/canton/16$' : 'AI',
    '^https://ld.admin.ch/canton/17$' : 'SG',
    '^https://ld.admin.ch/canton/18$' : 'GR',
    '^https://ld.admin.ch/canton/19$' : 'AG',
    '^https://ld.admin.ch/canton/20$' : 'TG',
    '^https://ld.admin.ch/canton/21$' : 'TI',
    '^https://ld.admin.ch/canton/22$' : 'VD',
    '^https://ld.admin.ch/canton/23$' : 'VS',
    '^https://ld.admin.ch/canton/24$' : 'NE',
    '^https://ld.admin.ch/canton/25$' : 'GE',
    '^https://ld.admin.ch/canton/26$' : 'JU',
    '^https://ld.admin.ch/country/CHE$' : 'CH'
}

In [35]:
df_g3['kurzname'] = df_g3['kurzname'].replace(kanton_dict_lindas.keys() , kanton_dict_lindas.values() , regex=True)
df_g3

,kurzname,jahr,anzahl
0,CH,2010,0
1,CH,2010,5514
2,CH,2011,0
3,CH,2011,20547
4,CH,2012,0
...,...,...,...
359,UR,2022,147
360,VD,2022,2151
361,VS,2022,1277
362,ZG,2022,104


In [36]:
df_g3_no_duplicates = df_g3.drop_duplicates()
df_g3_no_duplicates

,kurzname,jahr,anzahl
0,CH,2010,0
1,CH,2010,5514
2,CH,2011,0
3,CH,2011,20547
4,CH,2012,0
...,...,...,...
177,UR,2022,147
178,VD,2022,2151
179,VS,2022,1277
180,ZG,2022,104


In [37]:
df_g3_grouped = df_g3_no_duplicates.groupby(['kurzname', 'jahr'], as_index=False)['anzahl'].sum()
df_g3_grouped

,kurzname,jahr,anzahl
0,AG,2017,267
1,AG,2018,400
2,AG,2019,786
3,AG,2020,745
4,AG,2021,1036
...,...,...,...
164,ZH,2018,965
165,ZH,2019,1400
166,ZH,2020,1513
167,ZH,2021,2680


# 4te Abfrage: Photovoltaik

In [65]:
Photovoltaik_QUERY = '''
PREFIX schema: <http://schema.org/>

select distinct ?jahr ?kanton ?anlagen ?verguetung ?kW ?pro1000000Einwohner
where {
        SERVICE <https://lindas.admin.ch/query>
            {?s <https://energy.ld.admin.ch/sfoe/bfe_ogd84_einmalverguetung_fuer_photovoltaikanlagen/Jahr> ?jahr.
            ?s <https://energy.ld.admin.ch/sfoe/bfe_ogd84_einmalverguetung_fuer_photovoltaikanlagen/Kanton> ?kanton.
            ?s <https://energy.ld.admin.ch/sfoe/bfe_ogd84_einmalverguetung_fuer_photovoltaikanlagen/AnzahlAnlagen> ?anlagen.
            ?s <https://energy.ld.admin.ch/sfoe/bfe_ogd84_einmalverguetung_fuer_photovoltaikanlagen/VerguetungCHF> ?verguetung.
            ?s <https://energy.ld.admin.ch/sfoe/bfe_ogd84_einmalverguetung_fuer_photovoltaikanlagen/InstallierteLeistungkW> ?kW.
            ?s <https://energy.ld.admin.ch/sfoe/bfe_ogd84_einmalverguetung_fuer_photovoltaikanlagen/InstallierteLeistungkWPro100000Einwohner> ?pro1000000Einwohner.
}}
'''

# Graph erstellen und SPARQL-Abfrage durchführen
g4 = Graph()
results_g4 = g4.query(Photovoltaik_QUERY)

result_g4_list = []
for row in results_g4:
    result_g4_list.append(row)

# Liste in ein Pandas DataFrame umwandeln
df_g4 = pd.DataFrame(result_g4_list, columns=['jahr', 'kurzname', 'anlagen', 'verguetung', 'kW', 'pro1000000Einwohner'])

In [66]:
# DataFrame anzeigen
df_g4

,jahr,kurzname,anlagen,verguetung,kW,pro1000000Einwohner
0,2014,https://ld.admin.ch/canton/19,52,620373.5,400.25,62
1,2016,https://ld.admin.ch/canton/14,111,1165199.6,1324.44,1640
2,2016,https://ld.admin.ch/canton/17,1181,11289964.6,12703.22,2528
3,2016,https://ld.admin.ch/canton/5,137,1613472.21,1765.89,1133
4,2016,https://ld.admin.ch/canton/11,410,3420105.8,3601.89,1337
...,...,...,...,...,...,...
264,2022,https://ld.admin.ch/canton/26,639,4514818.8,11327.62,15354
265,2022,https://ld.admin.ch/canton/18,705,5695020.6,14539.49,7221
266,2022,https://ld.admin.ch/canton/12,256,2353038.9,6062.65,3095
267,2022,https://ld.admin.ch/canton/13,858,6604512.4,17106.89,5844


In [67]:
df_g4['kurzname'] = df_g4['kurzname'].replace(kanton_dict_lindas.keys() , kanton_dict_lindas.values() , regex=True)
df_g4

,jahr,kurzname,anlagen,verguetung,kW,pro1000000Einwohner
0,2014,AG,52,620373.5,400.25,62
1,2016,SH,111,1165199.6,1324.44,1640
2,2016,SG,1181,11289964.6,12703.22,2528
3,2016,SZ,137,1613472.21,1765.89,1133
4,2016,SO,410,3420105.8,3601.89,1337
...,...,...,...,...,...,...
264,2022,JU,639,4514818.8,11327.62,15354
265,2022,GR,705,5695020.6,14539.49,7221
266,2022,BS,256,2353038.9,6062.65,3095
267,2022,BL,858,6604512.4,17106.89,5844


# 5te Abfrage: Gebäudeprogramm

In [68]:
Gebaeudeprogramm_QUERY = '''
PREFIX schema: <http://schema.org/>

select distinct ?jahr ?region ?betrag ?bereich
where {
        SERVICE <https://lindas.admin.ch/query>
            {?s <https://energy.ld.admin.ch/sfoe/bfe_ogd18_gebaeudeprogramm_auszahlungen/Jahr> ?jahr.
            ?s <https://energy.ld.admin.ch/sfoe/bfe_ogd18_gebaeudeprogramm_auszahlungen/region> ?region.
            ?s <http://schema.org/amount> ?betrag.
            ?s <https://energy.ld.admin.ch/sfoe/bfe_ogd18_gebaeudeprogramm_auszahlungen/massnahmenbereich> ?bereich.
}}
'''

# Graph erstellen und SPARQL-Abfrage durchführen
g5 = Graph()
results_g5 = g5.query(Gebaeudeprogramm_QUERY)

result_g5_list = []
for row in results_g5:
    result_g5_list.append(row)

# Liste in ein Pandas DataFrame umwandeln
df_g5 = pd.DataFrame(result_g5_list, columns=['jahr', 'kurzname', 'betrag', 'bereich'])

In [69]:
df_g5

,jahr,kurzname,betrag,bereich
0,2010,https://ld.admin.ch/country/CHE,43522889.95,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
1,2010,https://ld.admin.ch/country/CHE,0.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
2,2010,https://ld.admin.ch/country/CHE,10977986.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
3,2010,https://ld.admin.ch/country/CHE,3501873.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
4,2010,https://ld.admin.ch/country/CHE,92235671.95,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
...,...,...,...,...
7197,2022,https://ld.admin.ch/canton/4,65390.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
7198,2022,https://ld.admin.ch/canton/22,1616377.3,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
7199,2022,https://ld.admin.ch/canton/23,27137.95,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
7200,2022,https://ld.admin.ch/canton/9,103693.85,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...


In [70]:
df_g5['kurzname'] = df_g5['kurzname'].replace(kanton_dict_lindas.keys() , kanton_dict_lindas.values() , regex=True)
df_g5

,jahr,kurzname,betrag,bereich
0,2010,CH,43522889.95,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
1,2010,CH,0.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
2,2010,CH,10977986.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
3,2010,CH,3501873.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
4,2010,CH,92235671.95,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
...,...,...,...,...
7197,2022,UR,65390.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
7198,2022,VD,1616377.3,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
7199,2022,VS,27137.95,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...
7200,2022,ZG,103693.85,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...


In [74]:
import urllib.parse
def extract_last_part(url):
    last_part = url.split("/")[-1]
    return urllib.parse.unquote(last_part) # sonderzeichen anzeigen lassen
df_g5['bereich_kurz'] = df_g5['bereich'].apply(extract_last_part)
df_g5

,jahr,kurzname,betrag,bereich,bereich_kurz
0,2010,CH,43522889.95,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Haustechnik
1,2010,CH,0.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Indirekte Massnahmen
2,2010,CH,10977986.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Neubau
3,2010,CH,3501873.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Systemsanierung
4,2010,CH,92235671.95,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Total
...,...,...,...,...,...
7197,2022,UR,65390.0,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Indirekte_Massnahmen
7198,2022,VD,1616377.3,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Indirekte_Massnahmen
7199,2022,VS,27137.95,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Indirekte_Massnahmen
7200,2022,ZG,103693.85,https://energy.ld.admin.ch/sfoe/bfe_ogd18_geba...,Indirekte_Massnahmen


# PLZ zu Ortschaften

In [75]:
import requests
from bs4 import BeautifulSoup

# URL der Webseite mit der Tabelle
url = 'https://postleitzahlenschweiz.ch/tabelle/'

# SSL-Überprüfung deaktivieren. ok?
response = requests.get(url, verify=False)

# BeautifulSoup verwenden, um den HTML-Code zu analysieren (ich konnte inscripits nicht installieren
soup = BeautifulSoup(response.text, 'html.parser')

# Die Tabelle finden
table = soup.find('table', class_='tableizer-table')

# Daten aus der Tabelle extrahieren
data = []
for row in table.find_all('tr')[1:]:  # ohne Kopfzeile
    columns = row.find_all('td')
    postal_code = columns[0].text
    place = columns[1].text
    canton = columns[2].text
    data.append({'Postleitzahl': postal_code, 'Ort': place, 'Kanton': canton})

# Ausgabe der extrahierten Daten
for entry in data:
    print(entry)


C:\Users\sarak\Documents\GitHub\dv_hs23\venv\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'postleitzahlenschweiz.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'Postleitzahl': '1000', 'Ort': 'Lausanne', 'Kanton': 'Waadt'}
{'Postleitzahl': '1003', 'Ort': 'Lausanne', 'Kanton': 'Waadt'}
{'Postleitzahl': '1004', 'Ort': 'Lausanne', 'Kanton': 'Waadt'}
{'Postleitzahl': '1005', 'Ort': 'Lausanne', 'Kanton': 'Waadt'}
{'Postleitzahl': '1006', 'Ort': 'Lausanne', 'Kanton': 'Waadt'}
{'Postleitzahl': '1007', 'Ort': 'Lausanne', 'Kanton': 'Waadt'}
{'Postleitzahl': '1008', 'Ort': 'Jouxtens-Mézery', 'Kanton': 'Waadt'}
{'Postleitzahl': '1008', 'Ort': 'Prilly', 'Kanton': 'Waadt'}
{'Postleitzahl': '1009', 'Ort': 'Pully', 'Kanton': 'Waadt'}
{'Postleitzahl': '1010', 'Ort': 'Lausanne', 'Kanton': 'Waadt'}
{'Postleitzahl': '1011', 'Ort': 'Lausanne', 'Kanton': 'Waadt'}
{'Postleitzahl': '1012', 'Ort': 'Lausanne', 'Kanton': 'Waadt'}
{'Postleitzahl': '1015', 'Ort': 'Lausanne', 'Kanton': 'Waadt'}
{'Postleitzahl': '1018', 'Ort': 'Lausanne', 'Kanton': 'Waadt'}
{'Postleitzahl': '1020', 'Ort': 'Renens VD', 'Kanton': 'Waadt'}
{'Postleitzahl': '1022', 'Ort': 'Chavannes-près-Rene

In [77]:
df_plz = pd.DataFrame(data)
df_plz

,Postleitzahl,Ort,Kanton
0,1000,Lausanne,Waadt
1,1003,Lausanne,Waadt
2,1004,Lausanne,Waadt
3,1005,Lausanne,Waadt
4,1006,Lausanne,Waadt
...,...,...,...
4115,9652,Neu St. Johann,St. Gallen
4116,9655,Stein SG,St. Gallen
4117,9656,Alt St. Johann,St. Gallen
4118,9657,Unterwasser,St. Gallen


In [82]:
kanton_dict_kurz = {
    'Zürich': 'ZH',
    'Bern': 'BE',
    'Luzern': 'LU',
    'Uri': 'UR',
    'Schwyz': 'SZ',
    'Obwalden': 'OW',
    'Nidwalden': 'NW',
    'Glarus': 'GL',
    'Zug': 'ZG',
    'Freiburg': 'FR',
    'Solothurn': 'SO',
    'Basel-Stadt': 'BS',
    'Basel-Landschaft': 'BL',
    'Schaffhausen': 'SH',
    'Appenzell Ausserrhoden': 'AR',
    'Appenzell Innerrhoden': 'AI',
    'St. Gallen': 'SG',
    'Graubünden': 'GR',
    'Aargau': 'AG',
    'Thurgau': 'TG',
    'Tessin': 'TI',
    'Waadt': 'VD',
    'Wallis': 'VS',
    'Neuenburg': 'NE',
    'Genf': 'GE',
    'Jura': 'JU',
    'Fürstentum Liechtenstein': 'FL',
    'Büsingen': 'SH',
}

In [84]:
df_plz['Kanton'] = df_plz['Kanton'].replace(kanton_dict_kurz.keys() , kanton_dict_kurz.values() , regex=True)
df_plz

,Postleitzahl,Ort,Kanton
0,1000,Lausanne,VD
1,1003,Lausanne,VD
2,1004,Lausanne,VD
3,1005,Lausanne,VD
4,1006,Lausanne,VD
...,...,...,...
4115,9652,Neu St. Johann,SG
4116,9655,Stein SG,SG
4117,9656,Alt St. Johann,SG
4118,9657,Unterwasser,SG
